# MODFLOW API Quickstart

Detailed description

In [ ]:
import modflowapi
from modflowapi import Callbacks
import numpy as np
from pathlib import Path

Define paths to the modflow6 api shared library

In [ ]:
sim_ws = Path("../test_model")
dll = Path("./libmf6.dll")

### Create a callback function for adjusting model data

The callback function allows users to wrap function that updates the modflow model at different steps. The `modflowapi.Callbacks` object allows users to find the particular solution step that they are currently in. `modflowapi.Callbacks` includes:

   - `Callbacks.initalize`: the initialize callback sends loaded simulation data back to the user to make adjustments before the model begins solving. This callback only occurs once at the beginning of the MODFLOW6 simulation
   - `Callbacks.stress_period`: the stress period callback sends simulation data for each solution group to the user to make adjustments to stress packages at the beginning of each stress period.
   - `Callbacks.timestep_start`: the timestep_start callback sends simulation data for each solution group to the user to make adjustments to stress packages at the beginning of each timestep.
   - `Callbacks.timestep_end`: ?????
   - `Callbacks.iteration_start`: the iteration_start callback sends simulation data for each solution group to the user to make adjustments to stress packages at the beginning of each outer solution iteration.
   - `Callbacks.iteration_end`: the iteration_end callback sends simulation data for each solution group to the user to make adjustments to stress packages and check values of stress packages at the end of each outer solution iteration.
   
The user can use any of these callbacks within their callback function.

In [ ]:
def callback_function(sim, step):
    """
    A demonstration function that dynamically adjusts recharge
    and pumping in a modflow-6 model through the MODFLOW-API
    
    Parameters
    ----------
    sim : modflowapi.Simulation
        A simulation object for the solution group that is 
        currently being solved
    step : enumeration
        modflowapi.Callbacks enumeration object that indicates
        the part of the solution modflow is currently in.
    """
    ml = sim.test_model
    if step == Callbacks.initialize:
        print(sim.models)
    
    if step == Callbacks.stress_period:
        # adjust recharge for stress periods 7 through 12
        if sim.kper <= 6:
            rcha = ml.rcha_0
            spd = rcha.stress_period_data
            print(f"updating recharge: stress_period={ml.kper}")
            spd["recharge"] += 0.40 * sim.kper
        
    
    if step == Callbacks.timestep_start:
        print(f"updating wel flux: stress_period={ml.kper}, timestep={ml.kstp}")
        ml.wel.stress_period_data["flux"] -= ml.kstp * 1.5
    
    if step == Callbacks.iteration_start:
        # we can implement complex solutions to boundary conditions here!
        pass
    

The callback function is then passed to `modflowapi.run_model`

In [ ]:
modflowapi.run_model(dll, sim_ws, callback_function, verbose=False)